# Avaliação Continuada 2 (AC2) - Árvore de Decisão

Abaixo, iremos criar um modelo para classificar mensagens de spam a partir de um dataset obtido no Kaggle.

Para esse problema, iremos usar o método de classificação Árvore de Decisão.

## Imports

Primeiro, começamos a importar todas as bibliotecas que iremos precisar nesse notebook, apenas para simplificar mais tarde.

In [ ]:
#Leitura dos módulos
from __future__ import print_function

import pandas as pd
from pandas import Series,DataFrame


import numpy as np
import matplotlib.pyplot as plt

np.random.seed(1000)

## Dataset

Depois, inicializamos o nosso dataset que foi baixado do Kaggle e pode ser obtido a partir [desse link](https://www.kaggle.com/team-ai/spam-text-message-classification).